In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [7]:
def entropy(prop):
    return (-1*prop*np.log2(prop)) + (-1*(1-prop)*np.log2(1-prop))

In [10]:
entropy(0.5)

1.0

## 엔트로피의 성질
- 엔트로피의 최대값은 이산 확률변수의 클래스의 갯수에 따라 달라진다. 만약 이산 확률 변수가 가질 수 있는 클래스가 2**K 개이고 이산 확률 변수가 가질 수 있는 엔트로피의 최대값은 각 클래스가 모두 같은 확률을 가지는 때이다. 이 때 엔트로피의 값은 K이다.

## 엔트로피와 정보량
- 엔트로피는 확률변수가 담을 수 있는 정보량을 의미한다.
- 엔트로피가 0이면 확률변수는 결정론적이므로 확률 변수의 표본값이 변화하지 않는다. 즉 정보량이 없다. 따라서 확률 변수의 표본값을 관측한다고 해도 우리가 얻을 수 있는 정보는 없다. 
- 만약 문서에서 각 알파벳이 나올 확률이 다음과 같다고 가정하자(Dictvectorizer 기반)
    - {1/2,1/4,1/8,1/8}
- 이 4개의 문자를 그냥 보내기 위해서는 2자리수의 이진수를 사용해야 한다. 00,01,10,11
- 이 문자의 엔트로피를 계산해보자

In [14]:
def entropy2(prop1,prop2,prop3,prop4):
    assert (prop1 + prop2 + prop3 + prop4) == 1
    return (-1*prop1*np.log2(prop1)) + (-1*prop2*np.log2(prop2)) +\
(-1*prop3*np.log2(prop3)) + (-1*prop4*np.log2(prop4))

In [15]:
entropy2(1/2,1/4,1/8,1/8)

1.75

위와 같이 해당 문자들이 줄수있는 정보량은 최대 2에서 1.75에 해당한다. 따라서 원래의 이진법상에 해당하는 2가 아닌 1.75의 정보량이기 때문에 1000개의 문자를 보내야 한다면 2000개가 아닌 1750개만 보내도 된다.

## 표본 데이터가 주어진 경우
- 확률 변수 모형 즉, 이론적인 확률 밀도(질량)함수가 주어진 것이 아니라 데이터가 주어졌다면 확률 질량 함수를 추정하여서 엔트로피를 계산한다.
- 위에서 계속 다뤘다시피 엔트로피 계산을 위해서는 P(y)를 알아야 한다. 이를 어떻게 알 수 있을까 prior이다. 
- 예로 들어 데이터가 모두 80개이고 클래스 별로 40,40개가 각각 있으면? 0.5,0.5로 추정할 수 있다.

## 조건부 엔트로피
- 조건부 엔트로피는 상관관계가 있는 두 확률변수 X,Y가 있고 X의 값을 안다면 Y의 확률 변수가 가질 수 있는 정보의 양을 뜻한다.

In [26]:
condi_df = pd.DataFrame([[30,10],[10,30]])
condi_df.columns=['Y=0','Y=1']
condi_df.index=['X=0','X=1']
condi_df

,Y=0,Y=1
X=0,30,10
X=1,10,30


In [53]:
y_prior = sum(condi_df.iloc[:,0])/(sum(condi_df.iloc[:,0]) + sum(condi_df.iloc[:,1]))
y_prior

0.5

In [54]:
condi_entropy1 = entropy(3/4).copy()

In [55]:
condi_entropy2 = entropy(1/4).copy()

In [56]:
y_prior * condi_entropy1 + (1-y_prior)*condi_entropy2

0.8112781244591328

- 수학적 정의는 너무 복잡해보인다 위에서 제시한 함수에서도 수학적 정의를 알아채기 힘들 수도 있을 정도로 힘든데, 직관적으로 생각해보자.
- 이산형 공식을 알면 연속형 공식도 자연히 알게 되기 때문에 여기서는 이산형 문제만 다루도록 하겠다.
- 데이터의 수(prior)를 일단 구한다. 위의 식에서는 y_prior로 계산하게 된다. 즉, 클래스0에 속하는 데이터의 수와 클래스1에 속하는 데이터의 수에 ratio를 계산한다.
- 그 다음에는 어떤 데이터냐에 대한 문제로 들어간다. 이 단계로 인해서 조건부 엔트로피라고 불리는 건데, 데이터의 특성이 0이냐 1이냐로 나뉘게 되는 것이다. 데이터의 특성이 0일때의 엔트로피를 계산하고 데이터의 특성이 1일때의 엔트로피를 각각 계산한다. 이렇게 되면 이 자체가 조건부 엔트로피가 된다.
- 마지막으로 prior에 가중 평균을 내주면 된다.

## 크로스 엔트로피
- 주로 분류 문제의 목표값 분포와 예측값 분포를 비교하는데 사용된다.
- 예를 들어서 이진 분류문제에서는 Y는 0 또는 1이라는 값만 가질 수 있다. 또 예측값 y_hat 은 Y=1일 확률이 theta라고 하자. 모수 theta로 대표되는 Y의 분포와 모수로 theta_hat으로 대표되는 Y_hat의 분포라고 하면 크로스 엔트로피는 다음과 같이 추정된다.

- N : 총 데이터의 수
- theta : 목표값의 모수
- theta_hat : 예측값의 모수

In [71]:
from sklearn.datasets import load_iris
iris = load_iris()
dfX = pd.DataFrame(iris.data,columns=iris.feature_names)
dfy = pd.DataFrame(iris.target,columns=['y'])
df = pd.concat([dfX,dfy],axis=1)

In [74]:
df = df[df['y']!=2]

In [80]:
dfX = df.iloc[:,0:4]
dfy = df.iloc[:,-1]

In [82]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
model = LinearDiscriminantAnalysis().fit(dfX,dfy)
y_pred = model.predict(dfX)
y_true = dfy
y_true.shape,len(y_pred)

((100,), 100)

In [90]:
prop_feature = sum(y_true.values)/len(y_true.values)

In [91]:
prop_predict = sum(y_pred)/len(y_pred)

0.5

In [113]:
def cross_entropy(theta,theta_hat):
    return -theta*np.log2(theta_hat)-(1-theta)*np.log2(1-theta_hat)

In [114]:
cross_entropy(0.5,0.3)

1.1257693834979823

## 쿨백-라이블러 발산
- 크로스 엔트로피의 개량판이다.
- 목표 분포와 예측 분포의 차이를 정량화하는 방법의 하나이다.
- cross-entropy - object distribution's entropy
- 쿨백-라이블러 발산은 크로스 엔트로피에서 대상이 되는 분포의 엔트로피는 뺀 값이므로 상대 엔트로피라고도 한다. 그 값은 항상 양수이며 두 확률 분포가 완전히 같은 경우에만 0이 된다.

`Kullback-Leibler divergence = -p(y)*log2(q(y))-q(y)*log2(q(y)) - p(y)`

## 지니 불순도 (Gini impurity)
- 엔트로피와 유사한 개념으로 지니 불순도라는 것이 있다.
- 지니 불순도는 엔트로피처럼 확률 분포가 어느쪽에 치우쳐있는가를 재는 척도이다.
- 로그를 사용하지 않기 때문에 computation 문제에서 부담이 적다.